# FeatherFace Baseline Training and Evaluation

This notebook reproduces the original FeatherFace training process following the author's instructions.

## Overview
- Model: FeatherFace with MobileNetV1 0.25x backbone
- Dataset: WIDERFace (auto-download)
- Expected Results: 0.49M parameters, 90.8% mAP
- Uses original training scripts for faithful reproduction

## 1. Installation and Environment Setup

In [ ]:
# Setup paths - all paths are relative to the FeatherFace root directory
import os
import sys
from pathlib import Path

# Get the project root directory (parent of notebooks/)
PROJECT_ROOT = Path(os.path.abspath('..'))
print(f"Project root: {PROJECT_ROOT}")

# Change to project root for all operations
os.chdir(PROJECT_ROOT)
print(f"Working directory: {os.getcwd()}")

In [ ]:
# Install project in editable mode
!pip install -e .

# Verify imports work
try:
    from models.retinaface import RetinaFace
    from data import cfg_mnet, WiderFaceDetection
    print("✓ Imports successful")
except ImportError as e:
    print(f"✗ Import error: {e}")

In [ ]:
# Verify imports and check GPU
import torch
import torchvision
import cv2
import numpy as np
import matplotlib.pyplot as plt
import gdown
import requests
import zipfile
import tarfile
import json
from datetime import datetime

print(f"Python version: {sys.version}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

## 2. Dataset Download and Preparation

The dataset will be automatically downloaded when training starts. But we can prepare the directories.

In [ ]:
# Check and create data directories
import os
from pathlib import Path

# Create necessary directories
data_dir = Path('data/widerface')
weights_dir = Path('weights')
results_dir = Path('results')


# WIDERFace download links
WIDERFACE_GDRIVE_ID = '11UGV3nbVv1x9IC--_tK3Uxf7hA6rlbsS'
WIDERFACE_URL = f'https://drive.google.com/uc?id={WIDERFACE_GDRIVE_ID}'

for dir_path in [data_dir, weights_dir, results_dir]:
    dir_path.mkdir(parents=True, exist_ok=True)
    print(f"✓ Directory ready: {dir_path}")


def download_widerface():
    """Download WIDERFace dataset from Google Drive"""
    output_path ='..'/ data_dir / 'widerface.zip'
    
    if not output_path.exists():
        print("Downloading WIDERFace dataset...")
        print("This may take several minutes depending on your connection.")
        
        try:
            gdown.download(WIDERFACE_URL, str(output_path), quiet=False)
            print(f"✓ Downloaded to {output_path}")
        except Exception as e:
            print(f"❌ Download failed: {e}")
            print("Please download manually from:")
            print(f"  {WIDERFACE_URL}")
            return False
    else:
        print(f"✓ Dataset already downloaded: {output_path}")
    
    return True

# Download dataset
if download_widerface():
    print("\n✅ Dataset download complete!")
else:
    print("\n❌ Please download the dataset manually.")

In [ ]:
# Extract dataset
def extract_widerface():
    """Extract WIDERFace dataset"""
    zip_path = '..'/data_dir / 'widerface.zip'
    
    if not zip_path.exists():
        print("❌ Dataset zip file not found. Please download first.")
        return False
    
    # Check if already extracted
    if (data_dir / 'train' / 'label.txt').absolute().exists() and \
       (data_dir / 'val' / 'wider_val.txt').absolute().exists():
        print("✓ Dataset already extracted")
        return True
    
    print("Extracting dataset...")
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(data_dir)
        print("✓ Dataset extracted successfully")
        return True
    except Exception as e:
        print(f"❌ Extraction failed: {e}")
        return False

# Extract dataset
if extract_widerface():
    print("\n✅ Dataset ready for use!")
else:
    print("\n❌ Please extract the dataset manually.")

In [ ]:
# Verify dataset structure
def verify_dataset():
    """Verify WIDERFace dataset structure"""
    required_files = [
        data_dir / 'train' / 'label.txt',
        data_dir / 'val' / 'wider_val.txt'
    ]
    
    all_present = True
    for file_path in required_files:
        if file_path.absolute().exists():
            print(f"✓ Found: {file_path.absolute()}")
        else:
            print(f"✗ Missing: {file_path.absolute()}")
            all_present = False
    
    # Check for images
    for split in ['train', 'val']:
        img_dir = data_dir / split / 'images'
        if img_dir.exists():
            img_count = len(list(img_dir.glob('**/*.jpg')))
            print(f"✓ {split} images: {img_count} found")
        else:
            print(f"✗ {split} images directory not found")
            all_present = False
    
    return all_present

dataset_ready = verify_dataset()
print(f"\nDataset verification: {'PASSED ✅' if dataset_ready else 'FAILED ❌'}")

## 3. Download Pre-trained Weights

The model requires pre-trained MobileNetV1 0.25x weights.

In [ ]:
# Pre-trained weights info
PRETRAIN_FILENAME = 'mobilenetV1X0.25_pretrain.tar'
pretrain_path = weights_dir / PRETRAIN_FILENAME

print("=== Pre-trained Weights Download Instructions ===")
print(f"\nWeights should be placed at: {pretrain_path.absolute()}")
print("\nDownload from:")
print("https://drive.google.com/open?id=1oZRSG0ZegbVkVwUd8wUIQx8W7yfZ_ki1")
print(f"\nSave as: {pretrain_path.relative_to('.')}")

if pretrain_path.exists():
    print(f"\n✓ Pre-trained weights found: {pretrain_path.relative_to('.')}")
else:
    print(f"\n✗ Pre-trained weights not found. Please download manually.")

## 4. Model Configuration and Training Parameters

In [ ]:
# Training parameters from original repository
TRAIN_CONFIG = {
    'network': 'mobile0.25',
    'num_workers': 1,  # Adjust based on your system
    'lr': 1e-3,
    'momentum': 0.9,
    'save_folder': 'weights/',
    'resume_net': None,  # Will be set to pretrained weights
    'batch_size': 32,
    'num_epoch': 250,
    'gpu': True,
    'ngpu': 1,
    'pretrain': True
}

print("Training Configuration:")
for key, value in TRAIN_CONFIG.items():
    print(f"  {key}: {value}")

## 5. Training Process

We'll use the original train.py script with our configuration.

In [ ]:
# Prepare training command
import subprocess
import sys

# Build command arguments
train_args = [
    sys.executable, 'train.py',
    '--network', TRAIN_CONFIG['network'],
    '--num_workers', str(TRAIN_CONFIG['num_workers']),
    '--lr', str(TRAIN_CONFIG['lr']),
    '--momentum', str(TRAIN_CONFIG['momentum']),
    '--save_folder', TRAIN_CONFIG['save_folder'],
    '--batch_size', str(TRAIN_CONFIG['batch_size']),
    '--num_epoch', str(TRAIN_CONFIG['num_epoch'])
]

if TRAIN_CONFIG['gpu']:
    train_args.extend(['--gpu', '--ngpu', str(TRAIN_CONFIG['ngpu'])])

if TRAIN_CONFIG['pretrain']:
    train_args.append('--pretrain')

print("Training command:")
print(' '.join(train_args))

In [ ]:
# Option 1: Run training directly (recommended for full training)
# Uncomment to run:
# result = subprocess.run(train_args, capture_output=True, text=True)
# print(result.stdout)
# if result.stderr:
#     print("Errors:", result.stderr)

# Option 2: Show manual command for terminal execution
print("\n=== To train manually in terminal ===")
print("Navigate to project root and run:")
print(' '.join(train_args).replace(sys.executable, 'python'))

## 6. Model Evaluation on WIDERFace

After training completes, we evaluate the model using test_widerface.py

In [ ]:
# Check for trained model
import glob

# Find the latest checkpoint
checkpoints = sorted(glob.glob('weights/mobilenet0.25_*.pth'))
if checkpoints:
    latest_checkpoint = checkpoints[-1]
    print(f"Found checkpoint: {latest_checkpoint}")
else:
    print("No checkpoints found. Please train the model first.")
    latest_checkpoint = None

In [ ]:
# Evaluation parameters
EVAL_CONFIG = {
    'trained_model': latest_checkpoint or 'weights/mobilenet0.25_Final.pth',
    'network': 'mobile0.25',
    'confidence_threshold': 0.02,
    'top_k': 5000,
    'nms_threshold': 0.4,
    'keep_top_k': 750,
    'save_folder': 'results/',
    'gpu': True
}

print("Evaluation Configuration:")
for key, value in EVAL_CONFIG.items():
    print(f"  {key}: {value}")

In [ ]:
# Build evaluation command
eval_args = [
    sys.executable, 'test_widerface.py',
    '--trained_model', EVAL_CONFIG['trained_model'],
    '--network', EVAL_CONFIG['network'],
    '--confidence_threshold', str(EVAL_CONFIG['confidence_threshold']),
    '--top_k', str(EVAL_CONFIG['top_k']),
    '--nms_threshold', str(EVAL_CONFIG['nms_threshold']),
    '--keep_top_k', str(EVAL_CONFIG['keep_top_k']),
    '--save_folder', EVAL_CONFIG['save_folder']
]

if EVAL_CONFIG['gpu']:
    eval_args.append('--gpu')

print("Evaluation command:")
print(' '.join(eval_args))

In [ ]:
# Option to run evaluation
print("\n=== To evaluate manually in terminal ===")
print("Navigate to project root and run:")
print(' '.join(eval_args).replace(sys.executable, 'python'))

# The evaluation will generate prediction files in results/

## 7. Model Analysis

Let's analyze the model architecture and count parameters.

In [ ]:
# Load and analyze model
import torch
from models.retinaface import RetinaFace
from data import cfg_mnet

# Create model
net = RetinaFace(cfg=cfg_mnet, phase='test')

# Count parameters
def count_parameters(model):
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    return total_params, trainable_params

total, trainable = count_parameters(net)
print(f"Total parameters: {total:,} ({total/1e6:.2f}M)")
print(f"Trainable parameters: {trainable:,} ({trainable/1e6:.2f}M)")

# Expected: ~0.49M parameters

In [ ]:
# Analyze model architecture by module
print("\n=== Model Architecture Analysis ===")
for name, module in net.named_children():
    params = sum(p.numel() for p in module.parameters())
    print(f"{name}: {params:,} parameters ({params/1e6:.3f}M)")

## 8. Results Summary

After running the evaluation, compare with expected baseline results:

In [ ]:
# Expected baseline results
baseline_results = {
    'Model': 'FeatherFace (MobileNetV1 0.25x)',
    'Parameters': '0.49M',
    'WIDERFace Easy': '90.8%',
    'WIDERFace Medium': '88.2%',
    'WIDERFace Hard': '77.2%',
    'Average mAP': '85.4%'
}

print("=== Expected Baseline Results ===")
for metric, value in baseline_results.items():
    print(f"{metric}: {value}")

print("\n=== Your Results ===")
print("Check results/ directory for evaluation outputs")
print("Use evaluation tools to compute mAP scores")

## 9. Next Steps - FeatherFace V2

With baseline established, we can proceed to Phase 02 for FeatherFace V2 development:

1. **Architecture Optimizations**:
   - Replace standard convolutions with grouped/depthwise convolutions
   - Implement CBAM++ attention modules
   - Optimize FPN with lightweight operations

2. **Target Specifications**:
   - Parameters: 0.25M (50% reduction)
   - Performance: 92%+ mAP (1.2% improvement)
   - Maintain real-time inference speed

3. **Implementation Plan**:
   - Create new model variant in models/
   - Implement optimized modules in layers/
   - Train with enhanced augmentation
   - Fine-tune hyperparameters